In [1]:
import numpy as np
import math

Let's implement a small network to play with. The network has input layer of 128 units, a hidden layer of 16 units and a sigmoid output unit. We collect the activations $Wx + b$ into vectors $a$ and the results of activation functions (applied on these activations) into vectors $h$.

In [2]:
N = 128  # Input size
H = 16   # Hidden layer size
M = 1    # Output size

x_input = np.random.randn(N)

# first affine layer weights and biases
W1 = np.random.randn(N, H)
b1 = 0.05 * np.random.randn(H)
# second affine layer weights and biases
W2 = np.random.randn(H, M)
b2 = 0.05 * np.random.randn(M)

# In the example case there's actually no need to store the values a but here they are anyway
a1 = np.zeros(H)            # W1 dot x + b1
h1 = np.zeros(H)            # f(a1) (f = sigmoid in the following code)
a2 = np.zeros(M)            # W2 dot h1 + b2
h2 = np.zeros(M)            # f(a2)

dW1 = np.zeros((N, H))      # gradient for W1 etc.
db1 = np.zeros(H)
dW2 = np.zeros((N, M))
db2 = np.zeros(M)

Forward pass of a layer (pre-activation function value).

In [3]:
def affine_forward(x, W, b):
    out = np.dot(x, W) + b
    return out

The backward pass returns gradients `g_x`, `g_W` and `g_b`. The last two contain the values gradient descent uses in its updates. Gradient `g_x` is needed to proceed further in the network (it will be the `g_upstream` for the previous layer etc.).

The implementation consists of Numpy code for the chain rule applications.  Since `b`is not needed to compute its gradient (or any other gradients) it's not passed as an argument either.

In [4]:
def affine_backward(g_upstream, x, W):
    g_b = np.copy(g_upstream)
    g_W = np.outer(x, g_upstream)
    g_x = np.dot(g_upstream, W.T)
    return g_x, g_W, g_b

Couple of element-wise nonlinearities along with their backward passes. Recall that we store the values of these in variables $h$ while computing the forward pass. Hence, we can use them again while doing the backwards pass (to compute the gradient values).

In [6]:
def relu_forward(x):
    out = np.maximum(np.zeros(x.shape), x)
    return out

# h = layer output in forward pass
# since h comes from relu, we know that the partial derivatives are 1 for h[i] > 0, and 0 otherwise
def relu_backward(g_upstream, h):
    # np.where creates an array of ones and zeros (depending on the outcome of the given test)
    # multiplication * is the element-wise multiplication of two vectors
    # this may look a bit awkward but allows the use of vector operations in all computations
    g_x = g_upstream * np.where(h > 0, np.ones(h.shape), np.zeros(h.shape))
    return g_x

In [7]:
def sigmoid_forward(x):
    s = 1. / (1. + np.exp(-x))
    return s

# h = sigmoid output in forward pass
def sigmoid_backward(g_upstream, h):
    # elementwise products
    sig_g = h * (1. - h)
    return g_upstream * sig_g

Loss function (binary cross-entropy) and its differential. Note that the differential has no parameter for the "upstream gradient" simply because there is none when this function is used (you can think it's 1). This gives us the first upstream gradient to start from.

In [8]:
# We add a small number to divisors to avoid div by zero
# This is a standard trick in numeric computations involving divisions
my_eps = 1e-8

def cross_entropy(y_true, y_pred):
    if (y_true == 1):
        return - math.log(y_pred)
    else:
        return - math.log(1.0 - y_pred)

def cross_entropy_diff(y_true, y_pred):
    if (y_true == 1):
        my_divisor = y_pred
    else:
        my_divisor = (1.0 - y_pred)
    my_divisor += my_eps
    g_loss = -(1./my_divisor)
    return g_loss

The forward and backward passes are specialized to our small 2-layer network. They should be generalized to handle any number of layers. Also, the weights, biases and activation functions of each layer should be passed in as arguments (e.g. as components of some larger structure). The programmer has just been busy (lazy).

In [9]:
# Using sigmoids all over
# You can try the same using relus in the hidden layer
def forward_pass(x):
    a1 = affine_forward(x, W1, b1)
    h1 = sigmoid_forward(a1) 
    a2 = affine_forward(h1, W2, b2)
    h2 = sigmoid_forward(a2)
    return (a1, h1, a2, h2)

Backward pass starts from the cost computation (single element). It should be straightforward to extend it to a minibatch of elements (sum up and average). Similarly, this routine should be generalized for arbitrary (layered) networks, and the variables `h` and `W` should be passed in as parameters.

In [10]:
def backward_pass(y_true, y_pred):
    g_loss = cross_entropy_diff(y_true, y_pred[0]) # forward pass result (h2) is a single-element array
    # the initial upstream gradient
    g = np.array([g_loss])

    g = sigmoid_backward(g, h2) # using here h2 instead of y_pred so it looks more "general"
    # print('g after passing layer 2 activation:', g)
    g, dW2, db2 = affine_backward(g, h1, W2)
    # print('g after passing layer 2:', g)
    g = sigmoid_backward(g, h1)
    # print('g after passing layer 1 activation:', g)
    g, dW1, db1 = affine_backward(g, x_input, W1)
    # print('g after passing layer 1:', g)
    return (dW1, db1, dW2, db2)

One could also make the GD loop below into a routine of its own.

In [11]:
# GD loop
# one can execute this code cell several times to continue training
n_iters = 1000
# This is what our model should learn (from one random input)
y_true = 1.
lr = 0.01
first_iter = True

print("Training for", n_iters, "loops")
for i in range(n_iters):
    a1, h1, a2, h2 = forward_pass(x_input)
    # You could add similar print after each 100 epochs etc.
    # Or make the reporting frequency a parameter
    if first_iter:
        print("Model output before training = ", h2)
        loss = cross_entropy(y_true, h2[0])
        print("Loss before training =         ", loss)
        first_iter = False
        
    dW1, db1, dW2, db2 = backward_pass(1., h2)

    W1 -= (lr * dW1)
    b1 -= (lr * db1)
    W2 -= (lr * dW2)
    b2 -= (lr * db2)
    
print("Model output after training = ", h2)
loss = cross_entropy(y_true, h2[0])
print("Loss after training =         ", loss)


Training for 1000 loops
Model output before training =  [0.05835468]
Loss before training =          2.841215739184117
Model output after training =  [0.98809613]
Loss after training =          0.011975283772418993
